In [588]:
import pandas as pd 
import numpy as np
import sklearn 
import nltk
import tweepy #if I want to practice pulling tweets 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import dateutil
from tqdm import tqdm
import re
import os
import lxml

In [589]:
#Scrape SCOTUS website for opinion info 
HOME_PATH = "https://www.supremecourt.gov/opinions/slipopinion/21"
response = requests.get(HOME_PATH)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
response, type(soup) 

<!DOCTYPE html>
<html xmlns="http://www.w3.org/1999/xhtml">
 <head id="ctl00_ctl00_Head1">
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="txt/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <script src="/js/jquery-3.1.0.min.js" type="text/javascript">
  </script>
  <script src="/js/bootstrap.js" type="text/javascript">
  </script>
  <link href="/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <link href="/css/bootstrap.min.css" rel="Stylesheet" type="text/css"/>
  <link href="/css/bootstrap-theme.min.css" rel="Stylesheet" type="text/css"/>
  <link href="/styles/newBootStrap2.css" rel="stylesheet" type="text/css"/>
  <!-- HTML5 shim and Respond.js IE8 support of HTML5 elements and media queries -->
  <!--[if lt IE 9]>
          <script src="/js/html5shiv.js"></script>
          <script src="/js/respond.min.js"></script>
        <![endif]-->
  <!--[if lt IE 8]>
          <

(<Response [200]>, bs4.BeautifulSoup)

In [590]:
#extract div name of month class header from scotus website tables
month_div_name = 'panel-heading'
#determine how many months (rows) in the table for the year's opinions
month_list_tag = soup.find_all(attrs={"class": month_div_name})
print('The number of',month_div_name,'is equal to', len(month_list_tag))


The number of panel-heading is equal to 6


In [591]:
#Create table of the months 
table1 = soup.find(attrs={'class': 'table table-bordered'})
print(table1)


<table cellpadding="2" cellspacing="0" class="table table-bordered" style="text-align: left;">
<tr>
<th scope="col" style="text-align: center;" width="20">R-</th>
<th scope="col" style="text-align: center;" width="60">Date</th>
<th scope="col" style="text-align: center;" width="60">Docket</th>
<th scope="col" style="text-align: center;" width="400">Name</th>
<th scope="col" style="text-align: center;" width="60">Revised</th>
<th scope="col" style="text-align: center;" width="20">J.</th>
<th scope="col" style="text-align: center;" width="40">Pt.</th>
</tr>
<tr>
<td style="text-align: center;">15</td>
<td style="text-align: center;">3/04/22</td>
<td style="text-align: center; white-space: nowrap;">20-828</td>
<td><a href="/opinions/21pdf/20-828_5ie6.pdf" target="_blank" title="Section 1806(f) of the Foreign Intelligence Surveillance Act of 1978—providing a procedure under which a trial-level court or other authority may consider the legality of electronic surveillance conducted under FIS

In [592]:
#name columns for new table
headers = []
for i in table1.find_all('th'):
    title = i.text
    headers.append(title)


In [593]:
#Create table using headers as columns
cases2021 = pd.DataFrame(columns = headers)
print(cases2021)

Empty DataFrame
Columns: [R-, Date, Docket, Name, Revised, J., Pt.]
Index: []


In [606]:
#Iterate through table1 to find all of the table rows and create a table out of the cases found
for j in soup.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data]
    if row != []:
        length = len(cases2021)
        cases2021.loc[length] = row
            

            
#cases2021.loc[length] = row2

,R-,Date,Docket,Name,Revised,J.,Pt.
0,15,3/04/22,20-828,FBI v. Fazaga,,A,595/2
1,14,3/04/22,20-443,United States v. Tsarnaev,3/04/22,T,595/2
2,13,3/03/22,20-601,"Cameron v. EMW Women’s Surgical Center, P. S. C.",3/04/22,A,595/2
3,12,3/03/22,20-827,United States v. Zubaydah,,B,595/2
4,11,2/24/22,20-915,"Unicolors, Inc. v. H&M Hennes & Mauritz, L. P.",,B,595/2
5,10,1/24/22,19-1401,Hughes v. Northwestern Univ.,,SS,595/1
6,9,1/20/22,20-637,Hemphill v. New York,1/21/22,SS,595/1
7,8,1/13/22,21A244,NFIB v. OSHA,,PC,595/1
8,7,1/13/22,21A240,Biden v. Missouri,,PC,595/1
9,6,1/13/22,20-480,Babcock v. Kijakazi,,AB,595/1


In [625]:
case_links = []
for link in soup.find_all('a', 
                          attrs={'href': re.compile("/opinions/21")}):
    case_links.append(link.get('href'))
    # display the actual urls
    #print(link.get('href'))  
print(case_links)

['/opinions/21pdf/20-828_5ie6.pdf', '/opinions/21pdf/20-443_new_2d8f.pdf', '/opinions/21pdf/20-443diff_1q24.pdf', '/opinions/21pdf/20-601_new_g20h.pdf', '/opinions/21pdf/20-601diff_2034.pdf', '/opinions/21pdf/20-827_i426.pdf', '/opinions/21pdf/20-915_pol1.pdf', '/opinions/21pdf/19-1401_m6io.pdf', '/opinions/21pdf/20-637_new_6khn.pdf', '/opinions/21pdf/20-637diff_6468.pdf', '/opinions/21pdf/21a244_hgci.pdf', '/opinions/21pdf/21a240_d18e.pdf', '/opinions/21pdf/20-480_b97c.pdf', '/opinions/21pdf/21-588_c07d.pdf', '/opinions/21pdf/21-463_new_8o6b.pdf', '/opinions/21pdf/21-463diff_o6p1.pdf', '/opinions/21pdf/143orig_1qm1.pdf', '/opinions/21pdf/20-1668_new_n7io.pdf', '/opinions/21pdf/20-1668diff_m5o1.pdf', '/opinions/21pdf/20-1539_09m1.pdf']


In [631]:
cases2021['cases'] = (case_links)

ValueError: Length of values (20) does not match length of index (15)

In [ ]:
#collect all links for cases and append to table 

  


In [ ]:
cases2021

,R-,Date,Docket,Name,Revised,J.,Pt.
0,15,3/04/22,20-828,FBI v. Fazaga,,A,595/2
1,14,3/04/22,20-443,United States v. Tsarnaev,3/04/22,T,595/2
2,13,3/03/22,20-601,"Cameron v. EMW Women’s Surgical Center, P. S. C.",3/04/22,A,595/2
3,12,3/03/22,20-827,United States v. Zubaydah,,B,595/2
4,11,2/24/22,20-915,"Unicolors, Inc. v. H&M Hennes & Mauritz, L. P.",,B,595/2
5,10,1/24/22,19-1401,Hughes v. Northwestern Univ.,,SS,595/1
6,9,1/20/22,20-637,Hemphill v. New York,1/21/22,SS,595/1
7,8,1/13/22,21A244,NFIB v. OSHA,,PC,595/1
8,7,1/13/22,21A240,Biden v. Missouri,,PC,595/1
9,6,1/13/22,20-480,Babcock v. Kijakazi,,AB,595/1


print(table1)

In [ ]:
#Opinions online are slip opinions which may contain errors, the official bound volumes currently only exist up until the year 2014/2015
#On the SCOTUS site, and are lengthy PDFs covering all of the court's opinions for a specific term. 
#It is worth looking at whether or not it is better to try and scrape a single massive PDF, or rather to try and test out the concepts 
#Using the opinions that can be scraped at an individual level from the pages specific to the opinions from 2016-2021


#Full Volume Convention: https://www.supremecourt.gov/opinions/boundvolumes/575BV.pdf

In [ ]:
#Web crawler to organize opinions by year
